In [2]:
import os
import os.path
import sys
import pandas as pd
import numpy as np
from importlib import reload

In [3]:
LOCAL_PATH = 'data'
RAW_TRACE = 'tencent_game_na.pcapng'
TCP_TRACE_FEATURE_FILE = 'tencent_game_na_tcp_pkt.csv'
UDP_TRACE_FEATURE_FILE = 'tencent_game_na_udp_pkt.csv'
TCP_FLOW_FEATURE_FILE = 'youtube_tcp_flow.csv'
UDP_FLOW_FEATURE_FILE = 'youtube_udp_flow.csv'
# BUCKET_NAME = '' # replace with your bucket name
# KEY = '' # replace with your object key

In [4]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [5]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))

Conversion done
CPU times: user 33.2 ms, sys: 34.3 ms, total: 67.5 ms
Wall time: 6.82 s
Conversion done
CPU times: user 1.35 s, sys: 95.3 ms, total: 1.45 s
Wall time: 8.7 s


In [6]:
def to_csv(trace_df, extract_func, file, features, max_packets_per_flow):
    extract_func(trace_df, 1.0, upsampled=True, max_packets_per_flow=max_packets_per_flow)[features].to_csv(file, index=False)

In [8]:
from python import flow_feature
reload(flow_feature)

FEATURES = ['avg(pkt_len)', 'stddev(pkt_len)', 'fb_ratio', 'inter_arrival_time', 'pkt_count']

tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if tcp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.tcp_generate, os.path.join(LOCAL_PATH, TCP_FLOW_FEATURE_FILE), FEATURES, max_packets_per_flow=1000)
    elif udp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.udp_generate, os.path.join(LOCAL_PATH, UDP_FLOW_FEATURE_FILE), FEATURES, max_packets_per_flow=1000)
    else:
        raise

Max pkt per flow->1000: 100%|██████████| 65980/65980 [00:11<00:00, 5880.15it/s]
/Users/shuojiang/Desktop/GA/Trace-Preprocessing/trace-feature-selection/python/flow_feature.py:81: RuntimeWarning: invalid value encountered in double_scalars
  flow['avg(pkt_len)'] = flow['pkt_len'] / flow['pkt_count']
/Users/shuojiang/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/shuojiang/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/shuojiang/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Max pkt per flow->1000: 100%|██████████| 271/271 [00:00<00:00, 5420.57it/s]
